In [1]:
import tensorflow as tf
import keras
import numpy as np

from official.projects.movinet.modeling import movinet
from official.projects.movinet.modeling import movinet_model

def create_model():
    model_id = 'a0'
    resolution = 224

    tf.keras.backend.clear_session()

    backbone = movinet.Movinet(model_id=model_id)
    backbone.trainable = False

    # Set num_classes=600 to load the pre-trained weights from the original model
    model = movinet_model.MovinetClassifier(backbone=backbone, num_classes=2)
    model.build([1, 8, 224, 224, 3])

    return model

model = create_model()

model.load_weights('movinet_fine_tuned')

c:\Users\Evan Mazor\code\repos\bite-recognition\.venv310\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
c:\Users\Evan Mazor\code\repos\bite-recognition\.venv310\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.1 and is not supported. 
Some things might work, some things might not.
If you were to en

In [22]:
bite_clip = np.fromfile('bite_clip.raw', dtype=np.float32).reshape(1, 8, 224, 224, 3)
print(bite_clip.shape)
print(model.predict(bite_clip))

(1, 8, 224, 224, 3)
1/1 [==============================] - 27s 27s/step
[[-1.1264563  1.172403 ]]


In [23]:
def preprocess_frame(frame):
    # Resize the frame to the input size expected by the MoViNet model
    frame_resized = cv2.resize(frame, (224, 224))  # Adjust size if your model uses different dimensions
    # Normalize the frame (0-255 to 0-1)
    frame_normalized = frame_resized / 255.0
    return frame_normalized

def get_clips_from_stream(cap, num_frames=8, stride=8):
    frames = []
    clip_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)

        clip_count += 1
        if clip_count % stride == 0:
            processed_frame = preprocess_frame(frame)
            frames.append(processed_frame)

        # Show the real-time camera feed
        cv2.imshow('Camera Feed', frame)

        # Once we've gathered enough frames for one clip
        if len(frames) == num_frames:
            yield np.array(frames)
            frames = []  # Reset for the next clip

        # Exit if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

In [24]:
import cv2

# Initialize video capture (0 for the first camera, or provide a filename for video file)
cap = cv2.VideoCapture(0)

print('starting')

for clip in get_clips_from_stream(cap):
    # Add batch dimension and predict
    clip_input = np.expand_dims(clip, axis=0)  # Shape should be (1, 8, 224, 224, 3)
    print(clip_input.shape)

    # Predict with the model
    prediction = model.predict(clip_input)

    # Analyze the prediction (for binary classification, for instance)
    print(prediction[0])
    if prediction[0][0] < prediction[0][1]:  # Adjust threshold if necessary
        print("Action detected!")
    else:
        print("No action detected.")

    # You can add visualization or further processing here

# Release the capture when done
cap.release()
cv2.destroyAllWindows()

starting
(1, 8, 224, 224, 3)
1/1 [==============================] - 20s 20s/step
[ 1.1196219 -1.0239307]
No action detected.
(1, 8, 224, 224, 3)
1/1 [==============================] - 15s 15s/step
[ 0.40332037 -0.3020206 ]
No action detected.
(1, 8, 224, 224, 3)
1/1 [==============================] - 14s 14s/step
[ 0.1806604  -0.08135715]
No action detected.
(1, 8, 224, 224, 3)
1/1 [==============================] - 11s 11s/step
[ 0.5883975  -0.49146715]
No action detected.
(1, 8, 224, 224, 3)
1/1 [==============================] - 12s 12s/step
[-2.2525742  2.3821332]
Action detected!
(1, 8, 224, 224, 3)
1/1 [==============================] - 11s 11s/step
[ 0.73208445 -0.61878085]
No action detected.
